In [3]:
!python -m pip install pyTelegramBotAPI
!pip install selenium
!pip install bs4

  Using cached selenium-3.141.0-py2.py3-none-any.whl (904 kB)
  Using cached bs4-0.0.1-py3-none-any.whl


In [6]:
!pip install pyttsx3

  Using cached pyttsx3-2.90-py3-none-any.whl (39 kB)
  Using cached pypiwin32-223-py3-none-any.whl (1.7 kB)


In [46]:
!pip install numpy

In [12]:
import pyttsx3
import numpy as np
import pandas as pd
import telebot  # pip install pyTelegramBotAPI
import requests
from bs4 import BeautifulSoup  # pip install bs4
import selenium
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
TOKEN = '1795391045:AAFsmAKo_ResztkMR96grDLq-Ftyf1luPmY'
bot = telebot.TeleBot(TOKEN)
chat_id = '1756656744'

keyword = input('')
@bot.message_handler(commands = '텔비') # 음식 입력하면 실행
def send_shop_info(message):
    engine = pyttsx3.init()
    args = message.text.split()
    if args[1] == keyword: # /텔비야 {keyword} 맛집 알려줘
        
        driver = webdriver.Chrome('chromedriver.exe')

        driver.implicitly_wait(10)
        driver.get(f'https://www.diningcode.com/isearch.php?query={keyword}') # 김치찌개 맛집 검색 이동

        location = driver.find_element(By.XPATH,'//*[@id="contents"]/div[1]/div/img')

        action_loc = webdriver.ActionChains(driver)
        action_loc.move_to_element(location)
        action_loc.click() # 내 위치 업데이트
        action_loc.perform() #action함수 실행

        shop = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/a/span[1]').text

        score = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/p[3]')
        voice1 = f'내 주변 {keyword} 맛집검색결과, 맛집 점수 {score.text}. 상위 1순위 {shop}입니다. 이동경로를 탐색합니다'
        print(voice1)


        destination = driver.find_element(By.XPATH,'//*[@id="div_normal_nearby"]/ul/li[1]/a/span[3]')
        dt = destination.text # 목적지, 맛집주소
        # print(dt)


        driver.get('https://map.kakao.com/') # 맛집 길찾기 시작
        time.sleep(3)
        direction = driver.find_element(By.XPATH,'//*[@id="search.tab2"]/a')

        box = driver.find_element(By.XPATH,'/html/body/div[10]/div/div/img')
        action_box = webdriver.ActionChains(driver)
        action_box.move_to_element(box)
        action_box.click()
        action_box.perform()

        direction.click()
        startbox = driver.find_element(By.XPATH,'//*[@id="info.route.searchBox.list"]/div[1]/div[2]/input')
        startpoint = driver.find_element(By.XPATH,'//*[@id="info.route.waypointSuggest.input0"]')
        ds = '서울 강동구 양재대로 1377 (성내동)' # 대신it학원 주소

        action_start = webdriver.ActionChains(driver)
        action_start.move_to_element(startpoint)
        action_start.click()
        action_start.perform()

        startpoint.send_keys(ds, Keys.ENTER) # 출발지입력
        time.sleep(3)

        des = driver.find_element(By.XPATH,'//*[@id="info.route.waypointSuggest.input1"]')
        # des.click()
        des.send_keys(dt, Keys.ENTER) # 목적지입력
        time.sleep(3)
        car = driver.find_element(By.XPATH, '//*[@id="cartab"]')
        car.click()


        car_time = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[1]')
        print(car_time)

        print('car_t:',car_time.text)


        car_distance = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/p/span[2]')
        print('car_distance:',car_distance)
        car_info = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[6]/ul/li/div[1]/div/div[1]/div')
        print('car_info.text:',car_info.text)
        voice2 =f'{shop} 자차로 이동시 예상소요시간 {car_time.text}, 거리 {car_distance.text}, 예상 {car_info.text}입니다. 안전운전하세요'
        print(voice2)



        public_tr = driver.find_element(By.XPATH, '//*[@id="transittab"]')
        public_tr.click()
        pub_time = driver.find_element(By.XPATH, '//*[@id="info.flagsearch"]/div[5]/ul/li[1]/div[1]/span[1]')
        pub_info = driver.find_element(By.XPATH,'//*[@id="info.flagsearch"]/div[5]/ul/li[1]/div[1]/span[2]')
        voice3 = f'{shop} 대중교통 이용시 예상소요시간 {pub_time.text}, {pub_info.text}. 교통카드를 챙겨주세요.'
        print(voice3)

        work = driver.find_element(By.XPATH, '//*[@id="walktab"]')
        work.click()
        work_fast_time = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[2]/div[1]/div/p/span[1]')
        work_fast_dis = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[2]/div[1]/div/p/span[2]')
    #     work_slow_time = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[3]/div[1]/div/p/span[1]')
    #     work_slow_dis = driver.find_element(By.XPATH,'//*[@id="info.walkRoute"]/div[1]/ul/li[3]/div[1]/div/p/span[2]')
        voice4 = f'{shop} 도보 이용시 최단거리는 {work_fast_time.text}, {work_fast_dis.text} 입니다.'
        print(voice4)

        bicycle = driver.find_element(By.XPATH, '//*[@id="biketab"]')
        bicycle.click()
        bic_road_time = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/p/span[1]')
        bic_road_dis = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/p/span[2]')
        bic_roadinfo = driver.find_element(By.XPATH,'//*[@id="info.bikeRoute"]/div[1]/ul/li[1]/div[1]/div[1]/div')
        voice5 = f'{shop} 자전거 도로 이용시 예상 소요시간{bic_road_time.text}, 거리 {bic_road_dis.text}, {bic_roadinfo.text}'
        print(voice5)

        bot.reply_to(message,voice1)
        
        engine.setProperty('rate', 150)
        engine.save_to_file(voice1, "v1.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('v1.mp3', 'rb'))

        bot.reply_to(message,voice2)
        engine.setProperty('rate', 150)
        engine.save_to_file(voice2, "v2.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('v2.mp3', 'rb'))

        bot.reply_to(message,voice3)
        engine.setProperty('rate', 150)
        engine.save_to_file(voice3, "v3.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('v3.mp3', 'rb'))

        bot.reply_to(message,voice4)
        engine.setProperty('rate', 150)
        engine.save_to_file(voice4, "v4.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('v4.mp3', 'rb'))

        bot.reply_to(message,voice5)
        engine.setProperty('rate', 150)
        engine.save_to_file(voice5, "v5.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('v5.mp3', 'rb')

bot.polling()

SyntaxError: invalid syntax (<ipython-input-12-ef50b327a332>, line 159)

In [18]:
TOKEN = '1795391045:AAFsmAKo_ResztkMR96grDLq-Ftyf1luPmY'
bot = telebot.TeleBot(TOKEN)
chat_id = '1756656744'

@bot.message_handler(commands = ['텔비야'])
def send_weather(message):
    engine = pyttsx3.init()
    args = message.text.split()
    arg = message.text
    
    if '날씨' == args[1]:
        driver = webdriver.Chrome('chromedriver.exe')
        driver.implicitly_wait(10)
        driver.get('https://www.google.com/search?q=오늘날씨')
        time.sleep(3)
        weather= driver.find_element(By.XPATH,'//*[@id="wob_dc"]')
        weather_info = weather.text
        svg = driver.find_element(By.XPATH,'//*[@id="wob_gsvg"]')
        tmp_info = svg.text
        tmp_list = svg.text.split('\n')
        map_list = list(map(float, tmp_list))

        tmp_max = max(map_list)
        tmp_min = min(map_list)
        print(tmp_max, tmp_min)
        tmp_gap = float(tmp_max) - float(tmp_min)
        voice_tmp = f'오늘의 날씨정보는 {weather_info}, 최고기온 섭씨{tmp_max}도, 최저기온 섭씨{tmp_min}도, 일교차는{tmp_gap}도 입니다.'
        print(tmp_gap)

        bot.reply_to(message, voice_tmp)
        
        engine.setProperty('rate', 150)
        engine.save_to_file(voice_tmp, "tmp_v1.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('tmp_v1.mp3', 'rb'))

        if weather_info == '광역성 소나기':
            v_w_info1 = f'{weather_info}가 예정되어 있습니다. 휴대용 우산을 챙겨주세요.'
            bot.reply_to(message.v_w_info)
            engine.setProperty('rate', 150)
            engine.save_to_file(v_w_info, "w_info_voice.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('w_info_voice.mp3', 'rb'))
        elif weather_info == '비':
            v_w_info2 = f'{weather_info}가 예정되어 있습니다. 우산을 챙겨주세요'
            bot.reply_to(message.v_w_info2)
            engine.setProperty('rate', 150)
            engine.save_to_file(v_w_info2, "w_info_voice2.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('w_info_voice2.mp3', 'rb'))


        if tmp_gap > 7.5:
            voice_tmp_gap1 = f'오늘 일교차는 {tmp_gap}입니다. 얇은 옷을 겹쳐입거나 외투를 준비해주세요.'
            bot.reply_to(message, voice_tmp_gap1)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_tmp_gap1, "v_tmp_gap.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('v_tmp_gap.mp3', 'rb'))

        elif tmp_gap == 7.5:
            voice_tmp_gap2 = f'오늘 일교차는 {tmp_gap}입니다. 적정 일교차입니다.'
            bot.reply_to(message, voice_tmp_gap2)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_tmp_gap2, "v_tmp_gap2.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('v_tmp_gap2.mp3', 'rb'))
        else:
            voice_tmp_gap3 = f'오늘 일교차는 {tmp_gap}입니다. 일교차가 크지 않습니다.'
            bot.reply_to(message, voice_tmp_gap3)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_tmp_gap3, "v_tmp_gap3.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('v_tmp_gap3.mp3', 'rb'))


        tmp_avg = sum(map_list)/len(map_list)

        voice_tmp_avg = f'오늘의 평균 온도는 섭씨 {tmp_avg}도 입니다.'
        bot.reply_to(message, voice_tmp_avg)
        engine.setProperty('rate', 150)
        engine.save_to_file(voice_tmp_avg, "voice_tmp_avg.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('voice_tmp_avg.mp3', 'rb'))

        if tmp_avg >= 10 and tmp_avg <=16:
            voice_cl = f'기온별 옷차림 추천, {tmp_avg}도에는 자켓, 셔츠, 가디건, 간절기 야상, 살구색 스타킹이 적합합니다.'
            bot.reply_to(message, voice_cl)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl, "voice_cl.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl.mp3', 'rb'))
        elif tmp_avg >= 17 and tmp_avg <= 19:
            voice_cl2 = f'기온별 옷차림 추천, {tmp_avg}도에는 니트, 가디건, 후드티, 맨투맨, 청바지, 면바지, 슬랙스, 원피스가 적합합니다.'
            bot.reply_to(message, voice_cl2)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl2, "voice_cl2.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl2.mp3', 'rb'))
        elif tmp_avg >=20 and tmp_avg <=  22:
            voice_cl3 = f'기온별 옷차림 추천, {tmp_avg}도에는 긴팔티, 가디건, 후드티, 면바지 ,슬랙스, 스키니가 적합합니다'
            bot.reply_to(message, voice_cl3)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl3, "voice_cl3.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl3.mp3', 'rb'))
        elif tmp_avg >=23 and tmp_avg <= 26:
            voice_cl4 = f'기온별 옷차림 추천, {tmp_avg}도에는 반팔, 얇은 셔츠, 얇은 긴팔, 반바지, 면바지가 적합합니다.'
            bot.reply_to(message, voice_cl4)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl4, "voice_cl4.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl4.mp3', 'rb'))
        elif tmp_avg > 27:
            voice_cl5 = f'기온별 옷차림 추천, {tmp_avg}도에는 나시티, 반바지, 민소매, 원피스가 적합합니다.'
            bot.reply_to(message, voice_cl5)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl5, "voice_cl5.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl5.mp3', 'rb'))

        elif tmp_avg < 9:
            voice_cl6 = f'기온별 옷차림 추천, {tmp_avg}도에는 패딩점퍼, 코트, 야상, 목도리, 장갑, 히트텍이 적합합니다.'
            bot.reply_to(message, voice_cl6)
            engine.setProperty('rate', 150)
            engine.save_to_file(voice_cl6, "voice_cl6.mp3")
            engine.runAndWait()
            pyttsx3.init(driverName='sapi5') 
            time.sleep(3)
            bot.send_audio(chat_id=chat_id, audio=open('voice_cl6.mp3', 'rb'))
            
bot.polling()

24.0 15.0
9.0


In [17]:
import pyttsx3
import numpy as np
import pandas as pd
import telebot  # pip install pyTelegramBotAPI
import requests
from bs4 import BeautifulSoup  # pip install bs4
import selenium
from urllib import parse
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
TOKEN = '1795391045:AAFsmAKo_ResztkMR96grDLq-Ftyf1luPmY'
bot = telebot.TeleBot(TOKEN)
chat_id = '1756656744'

@bot.message_handler(commands = ['텔비'])
def send_news(message):
    engine = pyttsx3.init()
    driver = webdriver.Chrome('chromedriver.exe')
    driver.implicitly_wait(10)
    args = message.text.split()
    coivd = 'https://www.hankyung.com/tag/코로나'
    finance = 'https://www.hankyung.com/tag/주식'
    realestate = 'https://www.hankyung.com/tag/부동산'
    bitcoin = 'https://www.hankyung.com/tag/암호화폐'
    
    
    if args[1] == '코로나':
        driver.get(coivd)
        cov = driver.find_element(By.XPATH,'//*[@id="container"]/div[1]/div[1]/div[2]/ul[1]/li[2]/div[2]/h3/a').text
        cov_voice = f' 오늘의 코로나 뉴스 속보, {cov}'
        bot.reply_to(message, cov_voice)
        engine.setProperty('rate', 150)
        engine.save_to_file(cov_voice, "cov_voice.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('cov_voice.mp3', 'rb'))
    elif args[1] == '주식':
        driver.get(finance)
        fin = driver.find_element(By.XPATH,'//*[@id="container"]/div[1]/div[1]/div[2]/ul[1]/li[1]/div[2]/h3/a').text
        fin_voice = f' 오늘의 주식 뉴스 속보, {fin}'
        bot.reply_to(message, fin_voice)
        engine.setProperty('rate', 150)
        engine.save_to_file(fin_voice, "fin_voice.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('fin_voice.mp3', 'rb'))
    elif args[1] == '부동산':
        driver.get(realestate)
        real = driver.find_element(By.XPATH,'//*[@id="container"]/div[1]/div[1]/div[2]/ul[1]/li[1]/div[2]/h3/a').text
        real_voice = f' 오늘의 부동산 뉴스 속보, {real}'
        bot.reply_to(message, real_voice)
        engine.setProperty('rate', 150)
        engine.save_to_file(real_voice, "real_voice.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('real_voice.mp3', 'rb'))
    elif args[1] == '암호화폐' or args[1] == '비트코인':
        driver.get(bitcoin)
        coin = driver.find_element(By.XPATH,'//*[@id="container"]/div[1]/div[1]/div[2]/ul[1]/li[1]/div[2]/h3/a').text
        coin_voice = f' 오늘의 암호화폐 뉴스 속보, {coin}'
        bot.reply_to(message, coin_voice)
        engine.setProperty('rate', 150)
        engine.save_to_file(coin_voice, "coin_voice.mp3")
        engine.runAndWait()
        pyttsx3.init(driverName='sapi5') 
        time.sleep(3)
        bot.send_audio(chat_id=chat_id, audio=open('coin_voice.mp3', 'rb'))
        

bot.polling()